In [ ]:
! pip install fuzzywuzzy summa transformers sentencepiece nltk chardet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=9e0740d5f3d46340c691733239b749ff33b6c63d77e70fb8e418885ca62ff166
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 22.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
base_dir = "/content/drive/MyDrive"

In [ ]:
!unzip "/content/drive/MyDrive/model_checkpoints-20231210T172508Z-001.zip" -d "/content/drive/MyDrive"

Archive:  /content/drive/MyDrive/model_checkpoints-20231210T172508Z-001.zip
   creating: /content/drive/MyDrive/model_checkpoints/.ipynb_checkpoints/
  inflating: /content/drive/MyDrive/model_checkpoints/checkpoint_50.pt  


In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# from summa.summarizer import summarize
import torch
from transformers import AutoTokenizer, AutoModel
import transformers
from nltk.tokenize import sent_tokenize
import json
import pickle as pkl
import os
import pandas as pd
import chardet

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creating Paper and Review Embeddings

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
model1 = transformers.PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum").to(device)
tokenizer1 = transformers.PegasusTokenizer.from_pretrained("google/pegasus-xsum")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

In [ ]:
def get_encoding(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        # print(result["encoding"])
    return result['encoding']

def read_json(file_path):
    with open(file_path, 'rb') as file:
        data = file.read().decode('utf-8-sig')
        data = json.loads(data)
    return data

In [ ]:
def summarize(txt):
    chunk_size = 300
    input_chunks = [txt[i:i + chunk_size] for i in range(0, len(txt), chunk_size)]
    summaries = []
    for chunk in input_chunks:
        input_ids = tokenizer1.encode(chunk, return_tensors="pt", max_length=1024, truncation=True).to(device)
        summary_ids = model1.generate(input_ids, max_length=30, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary_text = tokenizer1.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary_text)
    final_summary = " ".join(summaries)
    return final_summary

In [ ]:
def paper_tokenization(paper_path):
    json_data = read_json(paper_path)
    # l=json_data['metadata']['sections']
    if 'sections' not in json_data:
        return None
    l = json_data['sections']
    sections=['abstract', 'introduction', 'related works', 'problem definition', 'idea', 'methodology', 'experiments', 'results', 'tables & figures', 'analysis', 'future work', 'overall', 'bibliography', 'external']
    d={}
    count_no_head = 0
    summary = ''
    for section in l:
        txt = section['text']
        summary_txt=txt
        while len(summary_txt.split())>200:
            summary_txt = summarize(summary_txt)
        summary += summary_txt

    final_summary = summary
    while len(final_summary.split())>250:
        final_summary=summarize(final_summary)
    print(len(final_summary))
    tokens = tokenizer.tokenize(final_summary)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = tokenizer.encode(final_summary, max_length=512, add_special_tokens=True, truncation=True, padding='max_length')
    input_ids = torch.tensor(input_ids).to(device)
    attention_mask = (input_ids != tokenizer.pad_token_id).long()
    input_ids = input_ids.unsqueeze(0)
    print(input_ids.shape)
    attention_mask = attention_mask.unsqueeze(0)
    outputs = model(input_ids, attention_mask=attention_mask)
    sentence_embeddings = outputs[0].squeeze()
    # d[k]=sentence_embeddings
    print(f"Paper embedding shape: {sentence_embeddings.shape}")
    # del sentence_embeddings
    del outputs
    del input_ids
    del token_ids
    del tokens
    del l
    del json_data
    return sentence_embeddings

In [ ]:
def review_tokenizer(review_path):
    with open(review_path, "r", encoding=get_encoding(review_path)) as file:
        content = file.read()

    tokens = tokenizer.tokenize(content)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = tokenizer.encode(content, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
    # Create the attention mask
    input_ids = torch.tensor(input_ids).to(device)
    attention_mask = (input_ids != tokenizer.pad_token_id).long()
    # input_ids, attention_mask = tokenizer.pad(input_ids, padding='max_length', max_length=512, return_tensors='pt', return_attention_mask=True)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    outputs = model(input_ids, attention_mask=attention_mask)
    sentence_embeddings = outputs[0].squeeze()
    print(f"Review embedding shape: {sentence_embeddings.shape}")
    del outputs
    del input_ids
    del token_ids
    del tokens
    del content
    del file
    return sentence_embeddings

In [ ]:
final_papers = {
   'paper_name': [],
   'tokenized_paper': [],
   'tokenized_reviews': []
}
# with open(os.path.join(base_dir, 'embeddings_ICLR_2018_quality_test_v1.pkl'), 'rb') as fr:
#    final_papers = pkl.load(fr)

print("Completed paper names:")
print(final_papers['paper_name'])

# init_path='Complete_Dataset'
# init_path = ''
# years = os.listdir(init_path)
years = ["ICLR_2018"]
# Change batch size as per your laptop's capacity
batch_size = 5
# This n is for self, to remember which iteration is going on, how many more are remaining, etc. Not used in code explicitly
n = 0
#n=? is over

df = pd.read_csv(os.path.join(base_dir, "data_latest_final2.csv"))

for year in years:
    path2 = os.path.join(base_dir, year)
    acceptance_list = os.listdir(path2)
    # for cat in acceptance_list:
    # Change this list when required
    for cat in ["Poster_Papers"]:
    # for cat in ["Rejected_Papers"]:
        path3 = os.path.join(path2, cat)
        papers = os.listdir(path3)
        papers.sort()
        # You will have to change indices here, based on number of papers in a folder.
        batch_papers = papers[batch_size * n : batch_size * n + batch_size]
        print(f"Current ID range {batch_size * n} to {batch_size * n + batch_size - 1}")
        # For last batch, you can use the following commented line and put appropriate indices
        # batch_papers = papers[495:496]
        for paper in batch_papers:
            print(f"\nCurrent Paper: {paper}")
            path4 = os.path.join(path3, paper)
            docs = os.listdir(path4)
            reviews = []
            count_pdf = len([x for x in docs if x[-4:] == '.pdf'])
            # Here, the metadata (Science Parse) file is used (I had named it {some text here}_new_metadata.json)
            count_json = len([x for x in docs if x[-17:] == 'new_metadata.json'])
            review_original_old = [f"{year}_{cat}_{paper}_({x})" for x in docs if x[-4:] == '.txt' and not x.__contains__("MetaReview")]
            review_original = []
            for name in review_original_old:
                name_tmp = name
                row = df[df["paper_name"].str.contains(name_tmp.split('(')[1][:-5])]
                if not row.empty:
                    review_original.append(name)
            count_txt = len(review_original)
            if count_json != 0 and count_txt != 0:
                for doc in docs:
                    if doc[-17:] == 'new_metadata.json':
                        tokenized_paper = paper_tokenization(os.path.join(path4, doc))
                    # if doc[-4:] == '.txt':
                        # tokenized_review = review_tokenizer(path4 + '/' + doc)
                        # reviews.append(tokenized_review)
                for review in review_original:
                  #  tokenized_review = review_tokenizer(path4 + '/' + review.split('(')[1][:-1])
                   print(f"Current review: {review}")
                   tokenized_review = review_tokenizer(os.path.join(path4, review.split('(')[1][:-1]))
                   reviews.append(tokenized_review)
                if tokenized_paper != None:
                  final_papers['paper_name'].append(f"{year}_{cat}_{paper}")
                  # final_papers['review_names'].append(review_original)
                  final_papers['tokenized_paper'].append(tokenized_paper)
                  # reviews = torch.stack(reviews)
                  # final_papers['tokenized_reviews'].append(reviews)
                  final_papers['tokenized_reviews'].append([(x, y) for x, y in zip(review_original, reviews)])
                  print(f"Completed {paper}")
                else:
                  print(f"Not Completed {paper}")
                del tokenized_paper
                del tokenized_review
            else:
               print(f"{paper} not available in csv")
            del review_original
            del docs
            del reviews
        del batch_papers
        del papers
    print(year)

with open(os.path.join(base_dir, 'Lexical Model', 'embeddings_ICLR_2018_quality_test.pkl'), 'wb') as f:
    pkl.dump(final_papers, f)

Completed paper names:
[]
Current ID range 0 to 4

Current Paper: B12Js_yRb
1130
torch.Size([1, 512])
Paper embedding shape: torch.Size([512, 768])
Current review: ICLR_2018_Poster_Papers_B12Js_yRb_(ICLR2018-B12Js_yRb-R2.txt)
Review embedding shape: torch.Size([512, 768])
Current review: ICLR_2018_Poster_Papers_B12Js_yRb_(ICLR2018-B12Js_yRb-R3.txt)
Review embedding shape: torch.Size([512, 768])
Current review: ICLR_2018_Poster_Papers_B12Js_yRb_(ICLR2018-B12Js_yRb-R1.txt)
Review embedding shape: torch.Size([512, 768])
Completed B12Js_yRb

Current Paper: B13njo1R-
1357
torch.Size([1, 512])
Paper embedding shape: torch.Size([512, 768])
Current review: ICLR_2018_Poster_Papers_B13njo1R-_(ICLR2018-B13njo1R--R2.txt)
Review embedding shape: torch.Size([512, 768])
Current review: ICLR_2018_Poster_Papers_B13njo1R-_(ICLR2018-B13njo1R--R1.txt)
Review embedding shape: torch.Size([512, 768])
Current review: ICLR_2018_Poster_Papers_B13njo1R-_(ICLR2018-B13njo1R--R3.txt)
Review embedding shape: torch.S

# Creating Paper Attended Review Representations

In [ ]:
with open(os.path.join(base_dir, 'Lexical Model', 'embeddings_ICLR_2018_quality_test.pkl'), 'rb') as f:
    final_papers = pkl.load(f)

In [ ]:
class Attention(nn.Module):
    def __init__(self, embed_dim, num_heads=1):
        super().__init__()
        self.attention_layer = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.attention_weights = None

    def forward(self, paper_embeddings, review_embeddings):
        attended_review_representations, self.attention_weights = self.attention_layer(paper_embeddings, review_embeddings, review_embeddings)
        return attended_review_representations

In [ ]:
# Example usage
attention = Attention(embed_dim=768)
attention = attention.to(device)

# Create input tensors from the given dataset
# paper_embeddings = torch.stack([torch.randn(512, 768), torch.randn(512, 768)])
# review_embeddings = torch.stack([torch.stack([torch.randn(512, 768), torch.randn(512, 768), torch.randn(512, 768)]), torch.stack([torch.randn(512, 768), torch.randn(512, 768), torch.randn(512, 768)])])

paper_embeddings = final_papers['tokenized_paper']
review_embeddings = final_papers['tokenized_reviews']

# print(paper_embeddings.size())
# print(review_embeddings.size())

# Compute the attended review representations for each paper
attended_review_representations = {
    'paper_review_name': [],
    'paper_attended_review': []
}
for i in range(len(paper_embeddings)):
    paper_embedding = paper_embeddings[i].unsqueeze(0).to(device)
    print(f"Paper: {paper_embedding.size()}")
    for j in range(len(review_embeddings[i])):
        print(torch.stack([review_embeddings[i][j][1]]).size())
        print(review_embeddings[i][j][0].split('(')[1][:-5], review_embeddings[i][j][0].split('(')[1])
        paper_review_embeddings = torch.stack([review_embeddings[i][j][1]]).to(device)
        attended_review_representation = attention(paper_embedding, paper_review_embeddings)
        attended_review_representations['paper_review_name'].append(review_embeddings[i][j][0].split('(')[1][:-5])
        attended_review_representations['paper_attended_review'].append(attended_review_representation)

# attended_review_representations = torch.stack(attended_review_representations)


Paper: torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
ICLR2018-B12Js_yRb-R2 ICLR2018-B12Js_yRb-R2.txt)
torch.Size([1, 512, 768])
ICLR2018-B12Js_yRb-R3 ICLR2018-B12Js_yRb-R3.txt)
torch.Size([1, 512, 768])
ICLR2018-B12Js_yRb-R1 ICLR2018-B12Js_yRb-R1.txt)
Paper: torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
ICLR2018-B13njo1R--R2 ICLR2018-B13njo1R--R2.txt)
torch.Size([1, 512, 768])
ICLR2018-B13njo1R--R1 ICLR2018-B13njo1R--R1.txt)
torch.Size([1, 512, 768])
ICLR2018-B13njo1R--R3 ICLR2018-B13njo1R--R3.txt)
Paper: torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
ICLR2018-B14TlG-RW-R3 ICLR2018-B14TlG-RW-R3.txt)
torch.Size([1, 512, 768])
ICLR2018-B14TlG-RW-R2 ICLR2018-B14TlG-RW-R2.txt)
torch.Size([1, 512, 768])
ICLR2018-B14TlG-RW-R1 ICLR2018-B14TlG-RW-R1.txt)
Paper: torch.Size([1, 512, 768])
torch.Size([1, 512, 768])
ICLR2018-B17JTOe0--R2 ICLR2018-B17JTOe0--R2.txt)
torch.Size([1, 512, 768])
ICLR2018-B17JTOe0--R3 ICLR2018-B17JTOe0--R3.txt)
torch.Size([1, 512, 768])
ICLR2018-B17JTOe0

In [ ]:
print(len(attended_review_representations['paper_attended_review']))
print(attended_review_representations['paper_attended_review'][-2].size())
print(len(attended_review_representations['paper_review_name']))

12
torch.Size([1, 512, 768])
12


In [ ]:
print(len(final_papers['tokenized_paper']))
print(len(final_papers['tokenized_reviews']))
sum = 0
for i in final_papers['tokenized_reviews']:
    sum += len(i)
print(sum)

4
4
12


In [ ]:
for i in attended_review_representations['paper_review_name']:
    print(i)
print(len(attended_review_representations['paper_review_name']))

ICLR2018-B12Js_yRb-R2
ICLR2018-B12Js_yRb-R3
ICLR2018-B12Js_yRb-R1
ICLR2018-B13njo1R--R2
ICLR2018-B13njo1R--R1
ICLR2018-B13njo1R--R3
ICLR2018-B14TlG-RW-R3
ICLR2018-B14TlG-RW-R2
ICLR2018-B14TlG-RW-R1
ICLR2018-B17JTOe0--R2
ICLR2018-B17JTOe0--R3
ICLR2018-B17JTOe0--R1
12


In [ ]:
with open(os.path.join(base_dir, 'Lexical Model', 'paper_attended_reviews_ICLR_2018_quality_test.pkl'), 'wb') as fw:
    pkl.dump(attended_review_representations, fw)

# Inference using trained Model

In [ ]:
with open(os.path.join(base_dir, 'Lexical Model', 'paper_attended_reviews_ICLR_2018_quality_test.pkl'), 'rb') as fr:
    final_paper_attended_reviews = pkl.load(fr)

In [ ]:
final_paper_attended_reviews.keys()

dict_keys(['paper_review_name', 'paper_attended_review'])

In [ ]:

labels_df = pd.read_csv(os.path.join(base_dir, 'data_latest_final2.csv'))
labels_df['paper_name'][0]

'ICLR2018-B11bwYgfM-R1_annotated'

In [ ]:
# Load labels from excel
labels_df = pd.read_csv(os.path.join(base_dir, 'data_latest_final2.csv'))
scores_df = pd.read_csv(os.path.join(base_dir, "Lexical Model", 'review_scores.csv'))

# Load the data
# data = torch.cat(final_paper_attended_reviews['paper_attended_review'])
data = []
# labels = torch.randn(159, 1)
# labels = torch.tensor(labels_df["informativeness_score"]).reshape(-1, 1)
labels = []

review_names = final_paper_attended_reviews['paper_review_name']

for i, name in enumerate(review_names):
    row = labels_df[labels_df["paper_name"].str.contains(name)]
    if not row.empty:
        informativeness_score = row["informativeness_score"].iloc[0]
        labels.append(informativeness_score)
        data.append(final_paper_attended_reviews['paper_attended_review'][i])

data = torch.cat(data)
labels = torch.tensor(labels).reshape(-1, 1)

print(data.shape)
print(labels.shape)

# Initialize an empty list to store the scores
data_scores = []

# Iterate over the review names
for i, name in enumerate(review_names):
    # Find the row in scores_df that contains the review name
    row = scores_df[scores_df["review_name"].str.contains(name)]

    # If the row is not empty
    if not row.empty:
        # Append the scores to the data_scores list
        data_scores.append(row.iloc[0].tolist()[:-1])  # Assuming scores start from the second column

# Create a dataframe from the data_scores list
data_scores_df = pd.DataFrame(data_scores, columns=scores_df.columns[:-1])

# Print the shape of the new dataframe
print(data_scores_df.shape)

torch.Size([12, 512, 768])
torch.Size([12, 1])
(12, 11)


In [ ]:
len(data)

12

In [ ]:
len(labels)

12

In [ ]:
len(data_scores)

12

In [ ]:
del final_paper_attended_reviews

In [ ]:
# # Define the input shape
# input_shape = (512, 768) # 512 represents sequence length, 768 is the dimension of your text embeddings

# hidden_units = 512

# class GRUWrapper(nn.Module):
#     def __init__(self, input_size, hidden_size, bidirectional=True, batch_first=True):
#         super().__init__()
#         self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, bidirectional=bidirectional, batch_first=batch_first)

#     def forward(self, x):
#         # Pass the input through the GRU layer
#         output, _ = self.gru(x)
#         # Only return the output tensor
#         return output

# # Create a sequential model
# # model = nn.Sequential(
# #     # Add a BiGRU layer
# #     # nn.GRU(input_size=input_shape[1], hidden_size=hidden_units, bidirectional=True, batch_first=True),
# #     GRUWrapper(input_size=input_shape[1], hidden_size=hidden_units),
# #     # Flatten the output
# #     nn.Flatten(),
# #     # Add a fully connected layer with one neuron
# #     nn.Linear(in_features=512*2*hidden_units, out_features=128)
# # )
# model = nn.Sequential(
#     # Add a BiGRU layer
#     GRUWrapper(input_size=input_shape[1], hidden_size=hidden_units),
#     # Add a dropout layer
#     nn.Dropout(0.25),
#     # Flatten the output
#     nn.Flatten(),
#     # Add three fully connected layers with 256, 128 and 64 neurons respectively
#     nn.Linear(in_features=512*2*hidden_units, out_features=256),
#     nn.ReLU(),
#     nn.Linear(in_features=256, out_features=128),
#     nn.ReLU(),
#     nn.Linear(in_features=128, out_features=64),
#     nn.ReLU(),
#     # Add a dropout layer
#     nn.Dropout(0.5),
#     # Add a fully connected layer with one neuron
#     nn.Linear(in_features=64, out_features=1)
# )

# model = model.to(device)

# # Define the loss function
# # criterion = nn.MSELoss()
# # criterion = nn.HuberLoss()

# # Define the optimizer
# # optimizer = torch.optim.Adam(model.parameters())
# # optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.01) # Apply L2 regularization

# # Train the model
# # num_epochs = 50

In [ ]:
class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, score_size):
        super(BiGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size*2 + score_size, 256)  # 2 for bidirection
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(dropout)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x, scores):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)  # 2 for bidirection
        out, _ = self.gru(x, h0)
        out = out[:, -1, :]
        out = torch.cat((out, scores), dim=1)  # Concatenate the GRU output and scores
        out = self.fc1(out)
        out = torch.relu(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = torch.relu(out)
        out = self.dropout2(out)
        out = self.fc3(out)
        out = torch.relu(out)
        out = self.dropout3(out)
        out = self.fc4(out)
        out = torch.sigmoid(out)
        return out


In [ ]:
# To be used
# Alternative arch
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, score_size):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size*2, 256)  # 2 for bidirection
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.fc5 = nn.Linear(32 + score_size, 16)
        self.bn5 = nn.BatchNorm1d(16)
        self.fc6 = nn.Linear(16, 1)

    def forward(self, x, scores):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)  # 2 for bidirection
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(x.device)  # 2 for bidirection
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        elu = torch.nn.ELU()
        out = self.fc1(out)
        out = self.bn1(out)
        out = elu(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out = elu(out)
        out = self.fc3(out)
        out = self.bn3(out)
        out = elu(out)
        out = self.fc4(out)
        out = self.bn4(out)
        out = elu(out)
        out = torch.cat((out, scores), dim=1)  # Concatenate the fc4 output and scores
        out = self.fc5(out)
        out = self.bn5(out)
        out = elu(out)
        out = self.fc6(out)
        # out = torch.sigmoid(out)
        return out

model = BiLSTM(768, 512, 3, 0.5, len(data_scores_df.columns)).to(device)

In [ ]:
# Load the state dictionary
model.load_state_dict(torch.load(os.path.join(base_dir, "Lexical Model", 'best_model_lexical_new1.pth')))
model.eval()

BiLSTM(
  (lstm): LSTM(768, 512, num_layers=3, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (bn4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=43, out_features=16, bias=True)
  (bn5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
# Assuming your data is a PyTorch tensor
data_tensor = data.float().to(device)  # replace 'data' with your data variable
data_scores_tensor = torch.tensor(data_scores_df.values).float().to(device)

# Pass the data to the model
with torch.no_grad():
    output = model(data_tensor, data_scores_tensor)

In [ ]:
for n, i, j in zip (final_paper_attended_reviews['paper_review_name'], output, labels):
    print(f"{n}\t{i}\t{j}")

# Without sigmoid using BiLSTM

ICLR2018-B12Js_yRb-R2	tensor([0.7011], device='cuda:0')	tensor([0.8026], dtype=torch.float64)
ICLR2018-B12Js_yRb-R3	tensor([0.7107], device='cuda:0')	tensor([0.8026], dtype=torch.float64)
ICLR2018-B12Js_yRb-R1	tensor([0.6957], device='cuda:0')	tensor([0.8621], dtype=torch.float64)
ICLR2018-B13njo1R--R2	tensor([0.7367], device='cuda:0')	tensor([0.7360], dtype=torch.float64)
ICLR2018-B13njo1R--R1	tensor([0.6923], device='cuda:0')	tensor([0.8644], dtype=torch.float64)
ICLR2018-B13njo1R--R3	tensor([0.6947], device='cuda:0')	tensor([0.7876], dtype=torch.float64)
ICLR2018-B14TlG-RW-R3	tensor([0.7231], device='cuda:0')	tensor([0.8274], dtype=torch.float64)
ICLR2018-B14TlG-RW-R2	tensor([0.7316], device='cuda:0')	tensor([0.8141], dtype=torch.float64)
ICLR2018-B14TlG-RW-R1	tensor([0.7155], device='cuda:0')	tensor([0.9122], dtype=torch.float64)
ICLR2018-B17JTOe0--R2	tensor([0.7672], device='cuda:0')	tensor([0.8619], dtype=torch.float64)
ICLR2018-B17JTOe0--R3	tensor([0.7298], device='cuda:0')	tens

# Testing for smallest and highest informativeness scores

In [ ]:
with open(os.path.join(base_dir,  'Lexical Model', 'paper_attended_reviews_ICLR_2018.pkl'), 'rb') as fr:
    final_paper_attended_reviews = pkl.load(fr)

In [ ]:
# Load labels from excel
labels_df = pd.read_csv(os.path.join(base_dir, 'Lexical Model', 'data_latest_final2.csv'))

In [ ]:
scores_df = pd.read_csv(os.path.join(base_dir, "Lexical Model", 'review_scores.csv'))

## Smallest

In [ ]:
import numpy as np

# Load the data
# data = torch.cat(final_paper_attended_reviews['paper_attended_review'])
data = []
# labels = torch.randn(159, 1)
# labels = torch.tensor(labels_df["informativeness_score"]).reshape(-1, 1)
labels = []

review_names = final_paper_attended_reviews['paper_review_name']

for i, name in enumerate(review_names):
    row = labels_df[labels_df["paper_name"].str.contains(name)]
    if not row.empty:
        informativeness_score = row["informativeness_score"].iloc[0]
        labels.append(informativeness_score)
        data.append(final_paper_attended_reviews['paper_attended_review'][i])

# data = torch.cat(data)
# labels = torch.tensor(labels).reshape(-1, 1)

# Initialize an empty list to store the scores
data_scores = []

# Iterate over the review names
for i, name in enumerate(review_names):
    # Find the row in scores_df that contains the review name
    row = scores_df[scores_df["review_name"].str.contains(name)]

    # If the row is not empty
    if not row.empty:
        # Append the scores to the data_scores list
        data_scores.append(row.iloc[0].tolist()[:-1])  # Assuming scores start from the second column

# Convert labels to a numpy array for easier manipulation
labels_np = np.array(labels)

# Get the indices of the 10 smallest informativeness scores
indices = labels_np.argsort()[:10]

# Use these indices to filter the data, data_scores, and labels
data =  torch.cat([data[i] for i in indices])
data_scores = [data_scores[i] for i in indices]
labels = torch.tensor([labels[i] for i in indices]).reshape(-1, 1)
# Create a dataframe from the data_scores list
data_scores_df = pd.DataFrame(data_scores, columns=scores_df.columns[:-1])

print(data.shape)
print(labels.shape)
print(data_scores_df.shape)

torch.Size([10, 512, 768])
torch.Size([10, 1])
(10, 11)


In [ ]:
# Assuming your data is a PyTorch tensor
data_tensor = data.float().to(device)  # replace 'data' with your data variable
data_scores_tensor = torch.tensor(data_scores_df.values).float().to(device)

# Pass the data to the model

with torch.no_grad():
    output = model(data_tensor, data_scores_tensor)

In [ ]:
for n, i, j in zip (final_paper_attended_reviews['paper_review_name'], output, labels):
    print(f"{n}\t{i}\t{j}")

ICLR2018-B13EC5u6W-R2	tensor([0.5941], device='cuda:0')	tensor([0.4462], dtype=torch.float64)
ICLR2018-B13EC5u6W-R3	tensor([0.5625], device='cuda:0')	tensor([0.4505], dtype=torch.float64)
ICLR2018-B13EC5u6W-R1	tensor([0.6780], device='cuda:0')	tensor([0.4566], dtype=torch.float64)
ICLR2018-B16_iGWCW-R2	tensor([0.6698], device='cuda:0')	tensor([0.4728], dtype=torch.float64)
ICLR2018-B16_iGWCW-R3	tensor([0.6741], device='cuda:0')	tensor([0.4782], dtype=torch.float64)
ICLR2018-B16_iGWCW-R1	tensor([0.6647], device='cuda:0')	tensor([0.4955], dtype=torch.float64)
ICLR2018-B16yEqkCZ-R2	tensor([0.6165], device='cuda:0')	tensor([0.5058], dtype=torch.float64)
ICLR2018-B16yEqkCZ-R3	tensor([0.6265], device='cuda:0')	tensor([0.5077], dtype=torch.float64)
ICLR2018-B16yEqkCZ-R1	tensor([0.7449], device='cuda:0')	tensor([0.5117], dtype=torch.float64)
ICLR2018-B1CEaMbR--R2	tensor([0.7300], device='cuda:0')	tensor([0.5214], dtype=torch.float64)


## Largest

In [ ]:
import numpy as np

# Load the data
# data = torch.cat(final_paper_attended_reviews['paper_attended_review'])
data = []
# labels = torch.randn(159, 1)
# labels = torch.tensor(labels_df["informativeness_score"]).reshape(-1, 1)
labels = []

review_names = final_paper_attended_reviews['paper_review_name']

for i, name in enumerate(review_names):
    row = labels_df[labels_df["paper_name"].str.contains(name)]
    if not row.empty:
        informativeness_score = row["informativeness_score"].iloc[0]
        labels.append(informativeness_score)
        data.append(final_paper_attended_reviews['paper_attended_review'][i])

# data = torch.cat(data)
# labels = torch.tensor(labels).reshape(-1, 1)

# Initialize an empty list to store the scores
data_scores = []

# Iterate over the review names
for i, name in enumerate(review_names):
    # Find the row in scores_df that contains the review name
    row = scores_df[scores_df["review_name"].str.contains(name)]

    # If the row is not empty
    if not row.empty:
        # Append the scores to the data_scores list
        data_scores.append(row.iloc[0].tolist()[:-1])  # Assuming scores start from the second column

# Convert labels to a numpy array for easier manipulation
labels_np = np.array(labels)

# Get the indices of the 10 smallest informativeness scores
indices = labels_np.argsort()[::-1][:10]

# Use these indices to filter the data, data_scores, and labels
data =  torch.cat([data[i] for i in indices])
data_scores = [data_scores[i] for i in indices]
labels = torch.tensor([labels[i] for i in indices]).reshape(-1, 1)
# Create a dataframe from the data_scores list
data_scores_df = pd.DataFrame(data_scores, columns=scores_df.columns[:-1])

print(data.shape)
print(labels.shape)
print(data_scores_df.shape)

torch.Size([10, 512, 768])
torch.Size([10, 1])
(10, 11)


In [ ]:
# Assuming your data is a PyTorch tensor
data_tensor = data.float().to(device)  # replace 'data' with your data variable
data_scores_tensor = torch.tensor(data_scores_df.values).float().to(device)

# Pass the data to the model
with torch.no_grad():
    output = model(data_tensor, data_scores_tensor)

In [ ]:
for n, i, j in zip (final_paper_attended_reviews['paper_review_name'], output, labels):
    print(f"{n}\t{i}\t{j}")

ICLR2018-B13EC5u6W-R2	tensor([0.8684], device='cuda:0')	tensor([0.8889], dtype=torch.float64)
ICLR2018-B13EC5u6W-R3	tensor([0.7170], device='cuda:0')	tensor([0.8861], dtype=torch.float64)
ICLR2018-B13EC5u6W-R1	tensor([0.7122], device='cuda:0')	tensor([0.8813], dtype=torch.float64)
ICLR2018-B16_iGWCW-R2	tensor([0.7298], device='cuda:0')	tensor([0.8758], dtype=torch.float64)
ICLR2018-B16_iGWCW-R3	tensor([0.7137], device='cuda:0')	tensor([0.8701], dtype=torch.float64)
ICLR2018-B16_iGWCW-R1	tensor([0.7271], device='cuda:0')	tensor([0.8677], dtype=torch.float64)
ICLR2018-B16yEqkCZ-R2	tensor([0.8401], device='cuda:0')	tensor([0.8674], dtype=torch.float64)
ICLR2018-B16yEqkCZ-R3	tensor([0.7112], device='cuda:0')	tensor([0.8672], dtype=torch.float64)
ICLR2018-B16yEqkCZ-R1	tensor([0.6839], device='cuda:0')	tensor([0.8659], dtype=torch.float64)
ICLR2018-B1CEaMbR--R2	tensor([0.7182], device='cuda:0')	tensor([0.8639], dtype=torch.float64)
